In [1]:
!pip install fastapi uvicorn requests supabase python-telegram-bot python-dotenv pyngrok nest_asyncio


In [29]:
!pip install supabase

In [2]:
from getpass import getpass

TELEGRAM_TOKEN = getpass("TELEGRAM_TOKEN: ")
SUPABASE_URL = getpass("SUPABASE_URL (https://...): ")
SUPABASE_KEY = getpass("SUPABASE_KEY: ")
OPENROUTER_KEY = getpass("OPENROUTER_KEY: ")

NGROK_AUTH = getpass("NGROK_AUTH_TOKEN (optional, press enter to skip): ")

with open('/content/secrets.env','w') as f:
    f.write(f"TELEGRAM_TOKEN={TELEGRAM_TOKEN}\n")
    f.write(f"SUPABASE_URL={SUPABASE_URL}\n")
    f.write(f"SUPABASE_KEY={SUPABASE_KEY}\n")
    f.write(f"OPENROUTER_KEY={OPENROUTER_KEY}\n")
    if NGROK_AUTH:
        f.write(f"NGROK_AUTH={NGROK_AUTH}\n")

print("✅ Saved secrets to /content/secrets.env")


TELEGRAM_TOKEN: ··········
SUPABASE_URL (https://...): ··········
SUPABASE_KEY: ··········
OPENROUTER_KEY: ··········
NGROK_AUTH_TOKEN (optional, press enter to skip): ··········
✅ Saved secrets to /content/secrets.env


In [39]:
%%writefile /content/main.py
import os
import requests
from dotenv import load_dotenv
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse
from supabase import create_client

# تحميل المتغيرات
load_dotenv("/content/secrets.env")

TELEGRAM_TOKEN = os.getenv("TELEGRAM_TOKEN")
OPENROUTER_KEY = os.getenv("OPENROUTER_KEY")
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

# إنشاء clients
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
TELEGRAM_API_URL = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}"

app = FastAPI()

@app.post("/webhook")
async def webhook(request: Request):
    data = await request.json()
    print("📩 Telegram update:", data)

    if "message" in data and "text" in data["message"]:
        chat_id = data["message"]["chat"]["id"]
        user_id = data["message"]["from"]["id"]
        user_message = data["message"]["text"]

        # 🔹 حفظ رسالة المستخدم في Supabase
        try:
            supabase.table("conversationmemory").insert({
                "sender": str(user_id),
                "recipient": str(chat_id),
                "message": user_message
            }).execute()
        except Exception as e:
            print("❌ Insert user message error:", e)

        # 🔹 استرجاع آخر 5 رسائل لهذا المستخدم
        memory_text = ""
        try:
            res = supabase.table("conversationmemory") \
                .select("sender,message,created_at") \
                .eq("sender", str(user_id)) \
                .order("created_at", desc=False) \
                .execute()
            memories = res.data if res and 'data' in res else []
            last_messages = memories[-5:]
            memory_text = "\n".join([f"{m['sender']}: {m['message']}" for m in last_messages])
        except Exception as e:
            print("❌ Fetch last messages error:", e)

        # 🔹 بناء البرومبت للـ AI
        prompt = f"""You are iTech AI Assistant. Be kind and professional.
Context (last messages):
{memory_text}

User: {user_message}

Reply briefly and helpfully about iTech.
"""

        # 🔹 استدعاء OpenRouter LLM
        ai_reply = "آسف، خدمة الذكاء الصناعي مش شغالة دلوقتي. جربي بعد شوية."
        try:
            response = requests.post(
                "https://openrouter.ai/api/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {OPENROUTER_KEY}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": "nvidia/nemotron-nano-9b-v2:free",
                    "messages": [
                        {"role": "system", "content": "You are a helpful iTech assistant."},
                        {"role": "user", "content": prompt}
                    ]
                },
                timeout=20
            )
            response.raise_for_status()
            res_json = response.json()
            print("✅ LLM Response:", res_json)
            ai_reply = res_json["choices"][0]["message"]["content"]

            # 🔹 حفظ رد AI في Supabase
            try:
                supabase.table("conversationmemory").insert({
                    "sender": "AI",
                    "recipient": str(chat_id),
                    "message": ai_reply
                }).execute()
            except Exception as e:
                print("❌ Insert AI response error:", e)

        except Exception as e:
            print("❌ LLM error:", e)
            print("🔴 Full response text:", getattr(response, "text", "No response text"))

        # 🔹 إرسال الرد للتيليجرام
        try:
            requests.post(f"{TELEGRAM_API_URL}/sendMessage", json={
                "chat_id": chat_id,
                "text": ai_reply
            })
        except Exception as e:
            print("❌ Telegram send error:", e)

    return JSONResponse(content={"ok": True})


Overwriting /content/main.py


In [42]:
from pyngrok import ngrok
from dotenv import load_dotenv
import os, time

load_dotenv('/content/secrets.env')
NGROK_AUTH = os.getenv('NGROK_AUTH')

if NGROK_AUTH:
    ngrok.set_auth_token(NGROK_AUTH)

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# تشغيل السيرفر
get_ipython().system_raw("uvicorn main:app --host 0.0.0.0 --port 8000 > /content/uvicorn.log 2>&1 &")
time.sleep(2)
!tail -n 10 /content/uvicorn.log


🌍 Public URL: NgrokTunnel: "https://e15eac1144d5.ngrok-free.app" -> "http://localhost:8000"
INFO:     Started server process [20612]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [43]:
import requests, os

TELEGRAM_TOKEN = os.getenv("TELEGRAM_TOKEN")
NGROK_URL = "https://e15eac1144d5.ngrok-free.app"

url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/setWebhook"
resp = requests.get(url, params={"url": f"{NGROK_URL}/webhook"})
print(resp.json())


{'ok': True, 'result': True, 'description': 'Webhook was set'}


In [15]:
!tail -n 50 /content/uvicorn.log


INFO:     Started server process [8261]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
📩 Telegram update: {'update_id': 212316237, 'message': {'message_id': 46, 'from': {'id': 5661638274, 'is_bot': False, 'first_name': 'Salma', 'last_name': 'Azoz', 'username': 'Salmaazoz', 'language_code': 'en'}, 'chat': {'id': 5661638274, 'first_name': 'Salma', 'last_name': 'Azoz', 'username': 'Salmaazoz', 'type': 'private'}, 'date': 1757702697, 'text': 'hey'}}
✅ LLM Response: {'id': 'gen-1757702697-8aexRT4eCg7km1MHFBQt', 'provider': 'Nvidia', 'model': 'nvidia/nemotron-nano-9b-v2:free', 'object': 'chat.completion', 'created': 1757702698, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': 'Hello! How can I assist you today? 😊\n', 'refusal': None, 'reasoning': 'Okay, the user just said "hey". That\'s a gre

In [41]:
!pkill -f ngrok
!pkill -f uvicorn
